In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv001.ib.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/notebooks


In [2]:
import sys, os
import numpy as np
import pandas as pd

sys.path.append('..')
%aimport mre_pinn

sys.path.append('../../param_search')
%aimport param_search
ps = param_search

Using backend: pytorch



# Training PINNs on the BIOQIC phantom dataset

The objective is to see if the best model identified so far on the FEM box data set can perform well on a more realistic and noisy dataset- the BIOQIC phantom dataset. I will use similar parameter ranges as the single-frequency 2D experiment, but I will try using helmholtz and heterogeneous PDEs as I'm not sure yet which will perform better on this data.

In [13]:
%autoreload
data, _ = mre_pinn.data.load_bioqic_dataset('../data/BIOQIC', 'phantom')
data

Loading ../data/BIOQIC/phantom_unwrapped_dejittered.mat
    __header__: <class 'bytes'>
    __version__: <class 'str'>
    __globals__: <class 'list'>
    info: <class 'numpy.ndarray'> (1, 1) [('dx_m', 'O'), ('dy_m', 'O'), ('dz_m', 'O'), ('frequencies_Hz', 'O'), ('index_description', 'O'), ('size', 'O')]
    magnitude: <class 'numpy.ndarray'> (80, 128, 25, 8, 3, 8) uint16
    phase_unwrap_noipd: <class 'numpy.ndarray'> (80, 128, 25, 8, 3, 8) float64
Loading ../data/BIOQIC/phantom_elastogram.npy
     <class 'numpy.ndarray'> (8, 128, 80, 25) complex128
Loading ../data/BIOQIC/phantom_regions.npy
     <class 'numpy.ndarray'> (128, 80, 25) int64
Preprocessing data
Multi frequency 3D
<xarray.Dataset>
Dimensions:         (frequency: 8, component: 3, z: 25, x: 128, y: 80)
Coordinates:
  * frequency       (frequency) float64 30.0 40.0 50.0 60.0 70.0 80.0 90.0 100.0
  * component       (component) <U1 'z' 'y' 'x'
  * z               (z) float64 0.0 0.0015 0.003 0.0045 ... 0.033 0.0345 0.036
  * 

<xarray.Dataset>
Dimensions:         (frequency: 8, component: 3, z: 25, x: 128, y: 80)
Coordinates:
  * frequency       (frequency) float64 30.0 40.0 50.0 60.0 70.0 80.0 90.0 100.0
  * component       (component) <U1 'z' 'y' 'x'
  * z               (z) float64 0.0 0.0015 0.003 0.0045 ... 0.033 0.0345 0.036
  * x               (x) float64 0.0 0.0015 0.003 0.0045 ... 0.1875 0.189 0.1905
  * y               (y) float64 0.0 0.0015 0.003 0.0045 ... 0.1155 0.117 0.1185
    spatial_region  (x, y, z) int64 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 -1
Data variables:
    a               (frequency, x, y, z, component) float64 0.0 0.0 ... 46.88
    u               (frequency, x, y, z, component) complex128 (-525.22761241...
    mu              (frequency, x, y, z) complex128 0j 0j 0j 0j ... 0j 0j 0j 0j
    Ku              (frequency, x, y, z, component) complex128 (24.8871884338...
    Lu              (frequency, x, y, z, component) complex128 (45161740.9209...
    Mu              (frequency, x, y, z) complex128 (112.7046953284758+76.953...

In [35]:
# define the job template and name format

template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:1
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
source activate MRE-PINN

python ../../../train.py \\
    --data_root ../../../data/BIOQIC \\
    --data_name phantom \\
    --frequency {frequency} \\
    --xyz_slice {xyz_slice} \\
    --noise_ratio 0.0 \\
    --pde_name {pde_name} \\
    --omega0 {omega0} \\
    --n_layers {n_layers} \\
    --n_hidden {n_hidden} \\
    --activ_fn {activ_fn} \\
    --optimizer adam \\
    --learning_rate {learning_rate} \\
    --pde_loss_wt {pde_loss_wt} \\
    --data_loss_wt {data_loss_wt} \\
    --batch_size {batch_size} \\
    --n_iters {n_iters} \\
    --test_every {test_every} \\
    --save_every {save_every} \\
    --save_prefix {job_name}
'''
name = 'train_{frequency}_{xyz_slice}_{pde_name}_{omega0}_{n_hidden}_{activ_fn}'

# define the parameter space

param_space = ps.ParamSpace(
    frequency=[50, 60, 70, 80, 90, 100],
    xyz_slice=['2D'],
    pde_name=['helmholtz', 'hetero'],
    omega0=[16],
    n_layers=[5],
    n_hidden=[128, 256],
    activ_fn=['t'],
    learning_rate=1e-4,
    pde_loss_wt=1e-8,
    data_loss_wt=1,
    batch_size=128,
    n_iters=250000,
    test_every=1000,
    save_every=10000
)

len(param_space)

24

In [36]:
%autoreload
expt_name = '2022-09-23_phantom'

jobs = ps.submit(template, name, param_space, work_dir=expt_name, verbose=True)
jobs.to_csv(f'{expt_name}.jobs')

#import pandas as pd
#jobs = pd.read_csv(f'{expt_name}.jobs')

100%|██████████| 24/24 [00:02<00:00, 11.91it/s]
[11296099, 11296100, 11296101, 11296102, 11296103, 11296104, 11296105, 11296106, 11296107, 11296108, 11296109, 11296110, 11296111, 11296112, 11296113, 11296114, 11296115, 11296116, 11296117, 11296118, 11296119, 11296120, 11296121, 11296122]


In [37]:
status_cols = ['job_name', 'job_state', 'node_id', 'runtime', 'stdout', 'stderr']
ps.status(jobs)[status_cols] #.iloc[0].stderr

,job_name,job_state,node_id,runtime,stdout,stderr
job_id,,,,,,
11296099,train_50_2D_helmholtz_16_128_t,RUNNING,v003,20:35,NaN,NaN
11296100,train_50_2D_helmholtz_16_256_t,RUNNING,v008,20:34,NaN,NaN
11296101,train_50_2D_hetero_16_128_t,RUNNING,v008,16:42,NaN,NaN
11296102,train_50_2D_hetero_16_256_t,PENDING,(Priority),0:00,NaN,NaN
11296103,train_60_2D_helmholtz_16_128_t,PENDING,(Priority),0:00,NaN,NaN
11296104,train_60_2D_helmholtz_16_256_t,PENDING,(Priority),0:00,NaN,NaN
11296105,train_60_2D_hetero_16_128_t,PENDING,(Priority),0:00,NaN,NaN
11296106,train_60_2D_hetero_16_256_t,PENDING,(Priority),0:00,NaN,NaN
11296107,train_70_2D_helmholtz_16_128_t,PENDING,(Priority),0:00,NaN,NaN


In [34]:
ps.status(jobs)[status_cols].iloc[0].stdout

In [ ]:
metrics = ps.metrics(jobs)

# did all models train to 100k iterations?
assert (metrics.groupby('job_name')['iteration'].max() == 100e3).all()

param_cols = ['frequency', 'omega0', 'n_hidden', 'activ_fn'] # experimental parameters
index_cols = ['iteration', 'variable_name', 'spatial_frequency_bin', 'spatial_region'] # metric identifiers
metric_cols = ['mean_squared_abs_value', 'power_density', 'mean_abs_value'] # metric values

metrics = metrics.groupby(['job_name'] + param_cols + index_cols, sort=False)[metric_cols].mean().unstack(level=[6])

def metric_map(t):
    metric_name, var_name = t
    metric_name = {
        'mean_squared_abs_value': 'MSAV',
        'mean_abs_value': 'MAV',
        'power_density': 'SPD'
    }[metric_name]
    new_col_name = f'{var_name}_{metric_name}'
    new_col_name = new_col_name.replace('diff_MSAV', 'pred_MSAE')
    new_col_name = new_col_name.replace('f_sum_MSAV', 'PDE_MSAE')
    new_col_name = new_col_name.replace('diff_MAV', 'pred_MAD')
    return new_col_name

metrics.columns = [metric_map(t) for t in metrics.columns.to_flat_index()]

m = metrics.reset_index()
metrics

In [ ]:
m['u_pred_MSAE_rel'] = m['u_pred_MSAE'] / m['u_true_MSAV']

fig = ps.plot(
    m[m.iteration == 100e3].copy(),
    x=param_cols,
    y=['u_pred_MSAE', 'u_true_MSAV', 'u_pred_MSAE_rel'],
    height=2.5,
    width=3,
    legend=False,
    tight=True
)

There is a clear trend of lower wave field error at higher frequencies, but this mostly due to the higher frequency wave images having lower amplitude. We can correct for this by plotting the relative wave field error. In the relative error plot, we see the inverse trend: higher frequency wave images have higher relative error in the predicted wave field. This is probably related to the spectral bias of neural networks- it's easier for them to learn lower frequencies than higher ones.

There also is a slight trend with respect to the omega0 parameter. Omega0 of 16 appears slightly better than 8, and in some cases increasing to 32 improves performance over 16.

It is not at all clear if there is a trend with model width.

In [ ]:
fig = ps.plot(
    m[m.iteration == 100e3].copy(),
    x=param_cols,
    y=['u_pred_MSAE_rel'],
    hue=('frequency', 'omega0'),
    height=8,
    width=3,
    legend=True,
    tight=True
)
fig.savefig('2d_experiment_u_pred_MSAE_rel_By_omega0.png', dpi=200, bbox_inches='tight')

When grouping by both data frequency and model frequency (omega0), we see that it is more difficult to fit wave fields with higher activation frequency. However, increasing the model frequency can help compensate slightly, as the wave field error decreases with higher model spectral bias (especially for frequency 100 Hz).

It also seems like there is an interaction here with the model width. It seems harder to fit a width 512 model when the model frequency bias is lower, at least for the case of the 100 Hz wave image.

In [ ]:
bin_size = 1e4
m['iteration_bin'] = (m.iteration // bin_size) * bin_size
m[m.job_name == 'train_100_2D_hetero_8_512_t']

In [ ]:
# investigate spectral bias
import seaborn as sns

m['u_diff_SPD_rel'] = m['u_diff_SPD'] / m['u_true_SPD']

sns.set_palette('Blues_r', 10)

fig = ps.plot(
    m[(m.job_name == 'train_100_2D_hetero_8_512_t') & (m.spatial_frequency_bin != 'all')].copy(),
    x='iteration',
    y=['u_diff_SPD_rel'],
    hue=('spatial_frequency_bin'),
    height=7.5,
    width=4.5,
    legend=True,
    tight=True,
    plot_func=sns.lineplot,
)
for ax in fig.axes:
    ax.set_yscale('log')
    ax.set_ylim(1e-4, 1e1)
fig.savefig('2d_experiment_power_spectrum8.png', dpi=200, bbox_inches='tight')

fig = ps.plot(
    m[(m.job_name == 'train_100_2D_hetero_32_512_t') & (m.spatial_frequency_bin != 'all')].copy(),
    x='iteration',
    y=['u_diff_SPD_rel'],
    hue=('spatial_frequency_bin'),
    height=7.5,
    width=4.5,
    legend=True,
    tight=True,
    plot_func=sns.lineplot,
)
for ax in fig.axes:
    ax.set_yscale('log')
    ax.set_ylim(1e-4, 1e1)
fig.savefig('2d_experiment_power_spectrum32.png', dpi=200, bbox_inches='tight')

In [ ]:
m['mu_pred_MAD_rel'] = m['mu_pred_MAD'] / m['mu_true_MAV']

sns.set_palette('tab10')

fig = ps.plot(
    m,
    x=param_cols,
    y=['mu_pred_MAD', 'mu_pred_MAD_rel'],
    height=3.5,
    width=3,
    hue='spatial_region',
    legend=True,
    tight=True
)

Now we are analyzing the relative error in the predicted elasticity, per region. Reconstruction appears to be hardest at the lower frequencies. The performance was best using frequencies 80 or 90 and worst at frequency 50.

It's also notable here that higher model frequency bias appears to HURT elasticity reconstruction performance, even through it previously seemed to help fit the model to the wave field. It might be worth trying to use different model spectral biases for the wave field and elastogram models.

In [ ]:
fig = ps.plot(
    m,
    x=param_cols,
    y=['mu_pred_MAD', 'mu_pred_MAD_rel'],
    height=5,
    width=3,
    hue=('frequency', 'omega0'),
    legend=True,
    tight=True
)
fig.savefig('2d_experiment_mu_pred_MAD_by_freq.png', dpi=200, bbox_inches='tight')

In [ ]:
sns.set_palette('Greens_r', 5)

fig = ps.plot(
    m[(m.job_name == 'train_100_2D_hetero_8_256_s') & (m.spatial_region != 'all')].copy(),
    x='iteration',
    y=['mu_pred_MAD_rel'],
    hue=('spatial_region'),
    height=6.25,
    width=4.5,
    legend=True,
    tight=True,
    plot_func=sns.lineplot,
)
for ax in fig.axes:
    #ax.set_yscale('log')
    ax.set_ylim(0, 1)
    pass
fig.savefig('2d_experiment_mu_pred_MAD_8.png', dpi=200, bbox_inches='tight')

fig = ps.plot(
    m[(m.job_name == 'train_100_2D_hetero_32_256_s') & (m.spatial_region != 'all')].copy(),
    x='iteration',
    y=['mu_pred_MAD_rel'],
    hue=('spatial_region'),
    height=6.25,
    width=4.5,
    legend=True,
    tight=True,
    plot_func=sns.lineplot,
)
for ax in fig.axes:
    #ax.set_yscale('log')
    ax.set_ylim(0, 1)
    pass
fig.savefig('2d_experiment_mu_pred_MAD_32.png', dpi=200, bbox_inches='tight')